#### Tuning Linear Regression Hyperparameters 

In [21]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

import pandas as pd

In [22]:
# Import data
df = pd.read_csv('./source/dataset.csv', index_col=0)
X, y = df[['RNN', 'GRU']], df['Actual']

# Get 1000 samples 
X_train = X[:1000]
y_train = y[:1000]
X_test = X[1000:]
y_test = y[1000:]

In [23]:
search_space = {
    'n_estimators': Integer(100, 1000),
    'max_depth': Integer(3, 30),
    'min_samples_split': Integer(2, 20),
    'min_samples_leaf': Integer(1, 20),
    'max_features': Categorical(['auto', 'sqrt', 'log2']),
    'max_leaf_nodes': Integer(10, 1000, "log-uniform"),
    'min_impurity_decrease': Real(0.0, 1e-1),
    'bootstrap': Categorical([True, False]),
    'criterion': Categorical(['squared_error', 'absolute_error', 'friedman_mse', 'poisson'])  # Updated criterion values
}

# Create a RandomForestRegressor instance
rf_model = RandomForestRegressor(random_state=0)

# Custom scorer function to return RMSE
def rmse_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    return -np.sqrt(mean_squared_error(y, y_pred))  # Negative RMSE for maximization

# Set up the BayesSearchCV
bayes_search = BayesSearchCV(
    estimator=rf_model,
    search_spaces=search_space,
    n_iter=32,
    scoring=rmse_scorer,  # Using custom RMSE scorer
    n_jobs=-1,
    cv=10,
    random_state=42
)

# Perform the search
bayes_search.fit(X_train, y_train)

# Best parameters and score
best_params = bayes_search.best_params_
best_score = -bayes_search.best_score_  # Converting back to positive RMSE

print("Best Parameters:", best_params)
print("Best Score (RMSE):", best_score)


Best Parameters: OrderedDict([('bootstrap', False), ('criterion', 'friedman_mse'), ('max_depth', 6), ('max_features', 'sqrt'), ('max_leaf_nodes', 14), ('min_impurity_decrease', 0.0002966630673990501), ('min_samples_leaf', 6), ('min_samples_split', 15), ('n_estimators', 963)])
Best Score (RMSE): 0.024412547181323775


In [24]:
# Print Score of the base model on the test set RMSE
base_model = RandomForestRegressor(random_state=0)
base_model.fit(X_train, y_train)
y_pred = base_model.predict(X_test)
base_score = np.sqrt(mean_squared_error(y_test, y_pred))
print("Base Model Score (RMSE):", base_score)

# Print Score of the model on the test set
best_model = bayes_search.best_estimator_
y_pred = best_model.predict(X_test)
best_score = np.sqrt(mean_squared_error(y_test, y_pred))
print("Best Model Score (RMSE):", best_score)

Base Model Score (RMSE): 0.02614195217668621
Best Model Score (RMSE): 0.024969892256096034


#### Record

CV = 5

Best Parameters: OrderedDict([('bootstrap', True), ('criterion', 'absolute_error'), ('max_depth', 3), ('max_features', 'auto'), ('max_leaf_nodes', 1000), ('min_impurity_decrease', 0.0), ('min_samples_leaf', 1), ('min_samples_split', 20), ('n_estimators', 1000)])
Best Score (RMSE): 0.02415181020750989
Best Score with Testing Data (RMSE): 0.02479821937597947

---

CV = 10

Best Parameters: OrderedDict([('bootstrap', True), ('criterion', 'absolute_error'), ('max_depth', 30), ('max_features', 'auto'), ('max_leaf_nodes', 1000), ('min_impurity_decrease', 0.0), ('min_samples_leaf', 20), ('min_samples_split', 20), ('n_estimators', 1000)])

Best Score (RMSE): 0.02432574583285039

Best Score with Testing Data (RMSE): 0.024992002343046434

---

Best Parameters: OrderedDict([('bootstrap', False), ('criterion', 'friedman_mse'), ('max_depth', 6), ('max_features', 'sqrt'), ('max_leaf_nodes', 14), ('min_impurity_decrease', 0.0002966630673990501), ('min_samples_leaf', 6), ('min_samples_split', 15), ('n_estimators', 963)])

Best Score (RMSE): 0.024412547181323775

Best Score with Testing Data (RMSE): 0.024969892256096034

---

Best Parameters: OrderedDict([('bootstrap', False), ('criterion', 'friedman_mse'), ('max_depth', 6), ('max_features', 'sqrt'), ('max_leaf_nodes', 14), ('min_impurity_decrease', 0.0002966630673990501), ('min_samples_leaf', 6), ('min_samples_split', 15), ('n_estimators', 963)])

Best Score (RMSE): 0.024412547181323775

Best Score with Testing Data (RMSE): 0.024969892256096034